In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

In [2]:
df = pd.read_json("News_Category_Dataset_v3.json", orient = "records", lines = True)

## Spaltenanalyse

In [5]:
# Spaltenliste für spätere Analysen
column_list = []
for column in df.columns:
    column_list.append(column)

In [6]:
# Kompletter Datensatz
length_df = len(df)
amount_attributes = len(column_list)
print('Anzahl an Datensätzen: ', length_df)
print('Anzahl an Attributen: ', amount_attributes)

Anzahl an Datensätzen:  209527
Anzahl an Attributen:  6


In [7]:
# Spaltenübersicht
print('Existing columns: \n', column_list)

Existing columns: 
 ['link', 'headline', 'category', 'short_description', 'authors', 'date']


In [8]:
# Datentyp pro Spalte
print('Datentyp je Attribut: \n', df.dtypes)

Datentyp je Attribut: 
 link                         object
headline                     object
category                     object
short_description            object
authors                      object
date                 datetime64[ns]
dtype: object


## Analyse der Daten für jede Spalte

In [24]:
# Eindeutige Werte pro Spalte
count_of_unique_values = {}
for column in column_list:
    valueset = set(df[column])
    valueset = {x for x in valueset if pd.notna(x)}
    count_of_unique_values[column] = len(valueset)/length_df

In [36]:
# Funktion um die einzelnen Spalten zu analysieren
def _column_values(column):

    # Vorberechnung der eindeutigen Werte
    count_of_unique_elements = df[column].value_counts()

    # print Spaltenname und Anzahl eindeutiger Werte in der Spalte
    print(column)
    print('Anzahl unique values:', count_of_unique_values[column])

    # < 0.05 / section 4
    if count_of_unique_values[column] < 0.05:
        print('Es gibt wenig verschiedene Werte (> 5%)')

        # # plot values
        fig = px.bar(count_of_unique_elements, title = 'Anzahl Vorkommen einzelner Werte in Attribut "{}"'.format(column), labels = {'index': 'Werte', 'value': 'Anzahl'})
        fig.update_layout(showlegend = False)
        fig.show()
        
    # < 0.35 / section 3
    elif count_of_unique_values[column] < 0.35:
        print('mittendrin - to do')
    

    #  < 1 / section 1 und 2 
    elif count_of_unique_values[column] < 1:
        print('idee: sich wiederholende Werte anzeigen - tbd')
            
    # == 1 / uniqueIdentifier
    elif count_of_unique_values[column] == 1:
        print('Die Werte sind den Datensätzen eindeutig zuzuordnen (unique_identifier)')

In [30]:
# Verschiedene Kategorien und deren Anzahl
_column_values('link')

link
Anzahl unique values: 0.9998043211614732
idee: sich wiederholende Werte anzeigen - tbd


In [31]:
# Verschiedene Kategorien und deren Anzahl
_column_values('headline')

headline
Anzahl unique values: 0.9926930658101343
idee: sich wiederholende Werte anzeigen - tbd


In [32]:
# Verschiedene Kategorien und deren Anzahl
_column_values('category')

category
Anzahl unique values: 0.00020045149312499105
Es gibt wenig verschiedene Werte (> 5%)


In [33]:
# Verschiedene Kategorien und deren Anzahl
_column_values('short_description')

short_description
Anzahl unique values: 0.8925914082671923
idee: sich wiederholende Werte anzeigen - tbd


In [34]:
# Verschiedene Kategorien und deren Anzahl
_column_values('authors')

authors
Anzahl unique values: 0.1392135619753063
mittendrin - to do


In [35]:
# Verschiedene Kategorien und deren Anzahl
_column_values('date')

date
Anzahl unique values: 0.018565626387052742
Es gibt wenig verschiedene Werte (> 5%)
